<a href="https://colab.research.google.com/github/clinewton/ML-Group-Final-Task/blob/main/Group_7_Sentiment_Analysis_on_Amazon_Products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Consumer Reviews on Amazon Products**

Dataset Link: [Consumer Reviews on Amazon Products](https://www.kaggle.com/datafiniti/consumer-reviews-of-amazon-products)

In [3]:
!pip install fast_ml
# import nltk
# nltk.download('stopwords')

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import csv
import gensim
import matplotlib.pyplot as plt
import seaborn as sns
import string
from nltk.corpus import stopwords
import re

from fast_ml.model_development import train_valid_test_split
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GlobalMaxPool1D
from keras.callbacks import *

from tensorflow.keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
SEQ_LEN = 200
labels = ['Negative','Positive']

## **Data Preparation for Training**



1.   Add sentiment feature to the dataset based on the corresponding rating.
2.   Feature extraction to get the required featires for model training
3.   Split the dataset into 70% train set, 20% dev set and 10% test set
4.   Encoding the dataset.



In [6]:
df = pd.read_csv('/content/drive/MyDrive/ML Datasets/1429_1.csv',quoting=csv.QUOTE_MINIMAL,encoding='utf-8')

In [7]:
df.shape

(34660, 21)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34660 entries, 0 to 34659
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    34660 non-null  object 
 1   name                  27900 non-null  object 
 2   asins                 34658 non-null  object 
 3   brand                 34660 non-null  object 
 4   categories            34660 non-null  object 
 5   keys                  34660 non-null  object 
 6   manufacturer          34660 non-null  object 
 7   reviews.date          34621 non-null  object 
 8   reviews.dateAdded     24039 non-null  object 
 9   reviews.dateSeen      34660 non-null  object 
 10  reviews.didPurchase   1 non-null      object 
 11  reviews.doRecommend   34066 non-null  object 
 12  reviews.id            1 non-null      float64
 13  reviews.numHelpful    34131 non-null  float64
 14  reviews.rating        34627 non-null  float64
 15  reviews.sourceURLs 

In [9]:
df.describe()

,reviews.id,reviews.numHelpful,reviews.rating,reviews.userCity,reviews.userProvince
count,1.0,34131.000000,34627.000000,0.0,0.0
mean,111372787.0,0.630248,4.584573,NaN,NaN
std,NaN,13.215775,0.735653,NaN,NaN
min,111372787.0,0.000000,1.000000,NaN,NaN
25%,111372787.0,0.000000,4.000000,NaN,NaN
50%,111372787.0,0.000000,5.000000,NaN,NaN
75%,111372787.0,0.000000,5.000000,NaN,NaN
max,111372787.0,814.000000,5.000000,NaN,NaN


In [10]:
df['reviews.rating'].value_counts()

5.0    23775
4.0     8541
3.0     1499
1.0      410
2.0      402
Name: reviews.rating, dtype: int64

In [11]:
# '1' for a positive rating (> 3) and '0' for a negative rating (<3)
df['reviews.sentiment'] = [1 if rating > 3 else 0 for rating in df['reviews.rating']]

df.groupby('reviews.sentiment').nunique()

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
reviews.sentiment,,,,,,,,,,,,,,,,,,,,,
0,33,25,33,6,32,33,2,610,965,648,0,2,0,51,3,1943,2344,1971,0,0,2188
1,39,47,38,5,38,39,2,1029,1935,3819,1,2,1,77,2,11716,32315,18075,0,0,25081


In [12]:
# Extract the required features, 'reviews.text' and 'reviews.sentiment'
data = df[['reviews.text','reviews.sentiment']]
data.head(15)

,reviews.text,reviews.sentiment
0,This product so far has not disappointed. My c...,1
1,great for beginner or experienced person. Boug...,1
2,Inexpensive tablet for him to use and learn on...,1
3,I've had my Fire HD 8 two weeks now and I love...,1
4,I bought this for my grand daughter when she c...,1
5,This amazon fire 8 inch tablet is the perfect ...,1
6,"Great for e-reading on the go, nice and light ...",1
7,"I gave this as a Christmas gift to my inlaws, ...",1
8,Great as a device to read books. I like that i...,1
9,I love ordering books and reading them with th...,1


In [13]:
data.keys()

Index(['reviews.text', 'reviews.sentiment'], dtype='object')

In [14]:
# Remove null values
data  = data.dropna()
check = data[data['reviews.text'].isnull()]
check.head()

,reviews.text,reviews.sentiment


### Data Splitting and Sequencing
We split the data and then implement the Keras tokenizer as well as its pad_sequences method to transform our text data into 3D float data so that the neural networks will be able to be trained on it.

In [15]:
x_train, y_train, x_valid, y_valid, x_test, y_test = train_valid_test_split(data, 'reviews.sentiment', train_size=0.7, valid_size=0.2, test_size=0.1, method='random')

In [16]:
X_tr, Y_tr = x_train['reviews.text'].values,pd.get_dummies(y_train.values).values
X_val, Y_val = x_valid['reviews.text'].values,pd.get_dummies(y_valid.values).values
X_test, Y_test = x_test['reviews.text'].values,pd.get_dummies(y_test.values).values

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_tr)

In [18]:
x_train_seq = tokenizer.texts_to_sequences(X_tr)
x_valid_seq = tokenizer.texts_to_sequences(X_val)
x_test_seq = tokenizer.texts_to_sequences(X_test)

In [19]:
x_train_seq=pad_sequences(x_train_seq, maxlen=SEQ_LEN)
x_valid_seq=pad_sequences(x_valid_seq, maxlen=SEQ_LEN)
x_test_seq=pad_sequences(x_test_seq, maxlen=SEQ_LEN)

In [20]:
vocab = len(tokenizer.word_index)+1
vocab

12483

In [21]:
x_train_seq.shape

(24261, 200)

In [22]:
Y_tr.shape

(24261, 2)

## **Build the model**

In [23]:
model=Sequential()
model.add(Embedding(vocab,300,input_length=SEQ_LEN,trainable=True))
model.add(LSTM(128,return_sequences=True,dropout=0.2))
model.add(GlobalMaxPool1D())
model.add(Dense(64,activation='relu'))
model.add(Dense(2,activation='softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 300)          3744900   
_________________________________________________________________
lstm (LSTM)                  (None, 200, 128)          219648    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 3,972,934
Trainable params: 3,972,934
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["acc"])
early_stopping=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=3) # checks each and every epoch for changes. If present, model stops
checkpoint=ModelCheckpoint('best_model.h5',monitor="acc",mode='auto',save_best_only=True,verbose=1, save_freq=1)

history=model.fit(x_train_seq,Y_tr,batch_size=16,epochs=1,validation_data=(x_valid_seq,Y_val),verbose=1,callbacks=[early_stopping,checkpoint],validation_steps=1)

   1/1517 [..............................] - ETA: 52:16 - loss: 0.3776 - acc: 0.8750
Epoch 00001: acc improved from -inf to 0.87500, saving model to best_model.h5
   2/1517 [..............................] - ETA: 11:59 - loss: 0.3041 - acc: 0.9062
Epoch 00001: acc improved from 0.87500 to 0.90625, saving model to best_model.h5
   3/1517 [..............................] - ETA: 23:47 - loss: 0.3216 - acc: 0.8958
Epoch 00001: acc did not improve from 0.90625
   4/1517 [..............................] - ETA: 19:01 - loss: 0.2999 - acc: 0.9062
Epoch 00001: acc did not improve from 0.90625
   5/1517 [..............................] - ETA: 16:38 - loss: 0.2884 - acc: 0.9125
Epoch 00001: acc improved from 0.90625 to 0.91250, saving model to best_model.h5
   6/1517 [..............................] - ETA: 16:12 - loss: 0.2801 - acc: 0.9167
Epoch 00001: acc improved from 0.91250 to 0.91667, saving model to best_model.h5
   7/1517 [..............................] - ETA: 15:41 - loss: 0.2528 - acc:

### **Model Validation**

In [28]:
model=keras.models.load_model('best_model.h5')

In [29]:
test_loss, test_acc = model.evaluate(x_test_seq, Y_test, verbose=2)
print('Model Accuracy:\t', test_acc)

109/109 - 10s - loss: 0.2114 - acc: 0.9331
Model Accuracy:	 0.9330640435218811


In [30]:
predictions = model.predict(x_test_seq)
print(pred for pred in predictions)

<generator object <genexpr> at 0x7f5983af2f50>


### **Sample Testing**

In [31]:
sample_comment=['Amazing product']
sample_token = tokenizer.texts_to_sequences(sample_comment)
sample_token_pad = pad_sequences(sample_token, maxlen=100)
prediction = model.predict(sample_token_pad)
print(prediction)
labels[np.argmax(prediction)]

[[0.05651147 0.9434885 ]]


'Positive'